In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# STEP 1: Download Web Pages (1 to 50)
for i in range(1, 51):
    url = requests.get(f"https://books.toscrape.com/catalogue/page-{i}.html")
    with open(f"AllPages/page{i}.html", 'w', encoding='UTF-8') as f:
        f.write(url.text)
    print(f"page{i} Downloaded")

page1 Downloaded
page2 Downloaded
page3 Downloaded
page4 Downloaded
page5 Downloaded
page6 Downloaded
page7 Downloaded
page8 Downloaded
page9 Downloaded
page10 Downloaded
page11 Downloaded
page12 Downloaded
page13 Downloaded
page14 Downloaded
page15 Downloaded
page16 Downloaded
page17 Downloaded
page18 Downloaded
page19 Downloaded
page20 Downloaded
page21 Downloaded
page22 Downloaded
page23 Downloaded
page24 Downloaded
page25 Downloaded
page26 Downloaded
page27 Downloaded
page28 Downloaded
page29 Downloaded
page30 Downloaded
page31 Downloaded
page32 Downloaded
page33 Downloaded
page34 Downloaded
page35 Downloaded
page36 Downloaded
page37 Downloaded
page38 Downloaded
page39 Downloaded
page40 Downloaded
page41 Downloaded
page42 Downloaded
page43 Downloaded
page44 Downloaded
page45 Downloaded
page46 Downloaded
page47 Downloaded
page48 Downloaded
page49 Downloaded
page50 Downloaded


In [6]:
# STEP 2: Parse the pages & collect book links
category_soups = []
for i in range(1, 51):
    with open(f"AllPages/page{i}.html", encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        category_soups.append(soup)

In [8]:
book_id = 1
for soup in category_soups:
    books = soup.select(".product_pod")  # each book in the list
    for book in books:
        book_link = book.h3.a['href']
        url = requests.get(f"https://books.toscrape.com/catalogue/{book_link}")
        with open(f"AllBooks/BOOK{book_id}.html", 'w', encoding='UTF-8') as f:
            f.write(url.text)
        print(f"Book{book_id} Downloaded")
        book_id += 1

ALL BOOKS DOWNLOADED


In [10]:
# STEP 3: Parse individual book pages
book_soups = []
for i in range(1, 1001):
    with open(f"AllBooks/BOOK{i}.html", encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), "html.parser")
        book_soups.append(soup)

In [11]:
# STEP 4: Extract book details
books_data = []
for soup in book_soups:
    heading_section = soup.select_one(".product_main")
    
    title = heading_section.h1.text
    price = heading_section.select_one(".price_color").text.split("£")[1]
    availability_status = heading_section.select_one(".instock.availability").text.strip().split("(")[0]
    availability_count = heading_section.select_one(".instock.availability").text.strip().split("(")[1].split("a")[0]
    
    rating = heading_section.select_one(".star-rating")['class'][1]  # e.g., "Three"
    upc = soup.select_one(".table tr td").text
    category = soup.select(".breadcrumb li a")[2].text  # 3rd link in breadcrumb

    books_data.append([title, price, category, availability_status, availability_count, rating, upc])
        

In [14]:
# STEP 5: Save to DataFrame then CSV
BooksDataFrame = pd.DataFrame(books_data, columns=["Title", "Price", "Category", "Status", "Stock Count", "Rating", "UPC"])

In [16]:
BooksDataFrame

,Title,Price,Category,Status,Stock Count,Rating,UPC
0,A Light in the Attic,51.77,Poetry,In stock,22,Three,a897fe39b1053632
1,Tipping the Velvet,53.74,Historical Fiction,In stock,20,One,90fa61229261140a
2,Soumission,50.10,Fiction,In stock,20,One,6957f44c3847a760
3,Sharp Objects,47.82,Mystery,In stock,20,Four,e00eb4fd7b871a48
4,Sapiens: A Brief History of Humankind,54.23,History,In stock,20,Five,4165285e1663650f
...,...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,55.53,Classics,In stock,1,One,cd2a2a70dd5d176d
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,Sequential Art,In stock,1,Four,bfd5e1701c862ac3
997,A Spy's Devotion (The Regency Spies of London #1),16.97,Historical Fiction,In stock,1,Five,19fec36a1dfb4c16
998,1st to Die (Women's Murder Club #1),53.98,Mystery,In stock,1,One,f684a82adc49f011


In [18]:
BooksDataFrame.to_csv("BooksDataFrame.csv", index=False, encoding="utf-8")